In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler

from kan import *
torch.set_default_dtype(torch.float64)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [2]:
def treasury_data_retrieval():
    df = pd.read_csv('../data/us_treasury_rates.csv')
    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values(by='Date', ascending=True, inplace=True)
    df = df.reset_index(drop=True)

    return df

df = treasury_data_retrieval()
df = df.drop(columns='Date').dropna().reset_index(drop=True)
df

,1 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
0,3.31,3.72,4.07,4.32,4.45,4.60,4.55,4.56,4.35,4.26,4.14,4.38,4.15
1,3.58,3.83,4.09,4.33,4.48,4.66,4.62,4.66,4.45,4.36,4.24,4.47,4.24
2,3.55,3.78,4.09,4.31,4.43,4.58,4.49,4.52,4.34,4.28,4.21,4.54,4.33
3,3.57,3.83,4.16,4.33,4.52,4.61,4.50,4.52,4.36,4.31,4.25,4.59,4.40
4,3.56,3.81,4.14,4.32,4.50,4.60,4.42,4.45,4.25,4.17,4.10,4.45,4.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...
529,4.75,4.63,4.51,4.51,4.43,4.30,4.17,4.11,4.08,4.13,4.19,4.46,4.36
530,4.66,4.56,4.49,4.48,4.40,4.27,4.17,4.13,4.11,4.17,4.23,4.50,4.40
531,4.65,4.54,4.47,4.46,4.38,4.23,4.13,4.09,4.07,4.13,4.19,4.45,4.35
532,4.59,4.53,4.46,4.46,4.38,4.23,4.15,4.10,4.07,4.12,4.17,4.43,4.33


In [5]:
from sklearn.metrics import mean_squared_error 
total = 0
count = 0

for i in range(383, 533, 30):
    first = df.iloc[i:(i+30), :]
    second = df.iloc[(i+1):(i+31), :]
    print(mean_squared_error(first, second))
    total += mean_squared_error(first, second)
    count += 1

total/count

0.0019725641025641017
0.0015617948717948718
0.0034569230769230753
0.002224358974358975
0.0020569230769230773


np.float64(0.00225451282051282)